In [1]:
# Import functions #
import argparse,datetime,os,sys,time

try:
    import plotly.graph_objects as go
except:
    go = None  

if go is None:
    print("plotly is not installed")

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets

from tools import load_groupfile, query_api, filterIQM, merge_dfs, make_vio_plot, make_vio_plot_df

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [3]:
#define widgets
#modality_widget=widgets.Dropdown(
#    options=['bold', 'structural'],
#    description='Modality:',
#    disabled=False)


modality_widget=widgets.RadioButtons(
    options=['bold', 'structural'],
    description='Modality:',
    disabled=False
)

TR_min=widgets.FloatSlider(
    min=1.5,
    max=5,
    step=0.1,
    description='TR min:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)


TR_max=widgets.FloatSlider(
    min=1.5,
    max=4,
    step=0.1,
    description='TR max:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)



TE_min=widgets.FloatSlider(
    min=0,
    max=.05,
    step=0.001,
    description='TE min:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)


TE_max=widgets.FloatSlider(
    min=0,
    max=.05,
    step=0.001,
    description='TE max:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='',
    slider_color='white',
    color='black'
)


select_parameters=widgets.SelectMultiple(
    options=['TR_min', 'TR_max', 'TE_min', 'TE_max'],
    description='Parameters',
    disabled=False)

In [4]:
display(modality_widget)

RadioButtons(description='Modality:', options=('bold', 'structural'), value='bold')

In [5]:
modal=modality_widget.value
display(select_parameters)
"If empty, no parameter restrictions will applied"

SelectMultiple(description='Parameters', options=('TR_min', 'TR_max', 'TE_min', 'TE_max'), value=())

'If empty, no parameter restrictions will applied'

In [6]:
for i in select_parameters.value:
        display(eval(i))

FloatSlider(value=1.5, continuous_update=False, description='TR min:', max=5.0, min=1.5, readout_format='')

FloatSlider(value=1.5, continuous_update=False, description='TR max:', max=4.0, min=1.5, readout_format='')

FloatSlider(value=0.0, continuous_update=False, description='TE min:', max=0.05, readout_format='', step=0.001…

FloatSlider(value=0.0, continuous_update=False, description='TE max:', max=0.05, readout_format='', step=0.001…

In [7]:
to_filter=select_parameters.value
filter_list=[]
filter_dict={'TR_min': "TR >= {}".format(TR_min.value),
             'TR_max': "TR < {}".format(TR_max.value),
             'TE_min': "TE >= {}".format(TE_min.value),
             'TE_max': "TE < {}".format(TE_max.value)
            }

for item in to_filter:
    add_item=filter_dict.get(item)
    filter_list.append(add_item)

filter_list= ['TR > 1.9', 'TR < 3'] #whatcha
print(filter_list)


['TR > 1.9', 'TR < 3']


In [8]:
# Arguments #

# laziness helper
# here = os.path.dirname(os.path.abspath(os.path.realpath(__file__)))
here = %pwd
homedir = os.path.expanduser('~/OBIWAN/')
# path to input of local data from MRIQC on your own dataset 
#group_file = os.path.join(homedir,'DERIVATIVES', 'MRIQC', 'control_bold.tsv')
#group_file = os.path.join(homedir,'DERIVATIVES', 'MRIQC', 'obese_bold.tsv')
group_file = os.path.join(homedir,'DERIVATIVES', 'MRIQC', 'all_before_bold.tsv')
#group_file = os.path.join(homedir,'DERIVATIVES', 'MRIQC','afterPREPROC',  'group_bold.tsv')
#group_file = os.path.join(homedir,'DERIVATIVES', 'MRIQC', 'group_T1w.tsv')

# scan type to query the API for [bold, T1w, T2w]
modality = 'bold'
#modality = 'T1w'

# any scan parameters that you want to filter the API search results by
"""Current possible filters:
   Tesla, TE, TR
   NOTE: Only working as *and* right now!
"""


# IQM variables to visualize
#need to add separate IQMs for structural and functional
IQM_to_plot = ['fd_mean','fd_num','fd_perc']
#IQM_to_plot = ['aor','aqi','dvars_nstd', 'tsnr', 'snr']
               #tsnr', 'snr']
               #'cjv','cnr', 'efc', 'snr_gm', 'snr_csf', 'rpve_gm', 'fwhm_avg']
               #'aor','aqi','dvars_nstd', 'tsnr', 'snr']
                    #'cjv','cnr', 'efc', 'snr_gm', 'snr_csf', 'rpve_gm', 'fwhm_avg'] # 
                    #'dvars_std','dvars_vstd',
                    #'efc','fber','fd_mean','fd_num','fd_perc','fwhm_avg','fwhm_x','fwhm_y',
                    #'fwhm_z','gcor','gsr_x','gsr_y','snr','summary_bg_k','summary_bg_mad',
                    #'summary_bg_mean','summary_bg_median','summary_bg_n','summary_bg_p05',
                    #'summary_bg_p95','summary_bg_stdv','summary_fg_k','summary_fg_mad',
                    #'summary_fg_mean','summary_fg_median','summary_fg_n','summary_fg_p05',
                    #'summary_fg_p95','summary_fg_stdv','dummy_trs']
                    
                      # variable names we might want to list
    #qc_var_list = ["aor","aqi","cjv","cnr","dummy_trs","dvars_nstd","dvars_std","dvars_vstd","efc","fber","fber","fd_mean","fd_num","fd_perc","fwhm_avg","fwhm_avg","fwhm_x","fwhm_y","fwhm_z","gcor","gsr_x","gsr_y","icvs_csf","icvs_gm",
#"icvs_wm","inu_med","inu_range","qi_1","qi_2","rpve_csf","rpve_gm","rpve_wm","snr", "snr_csf","snr_gm","snr_total","snr_wm","snrd_csf","snrd_gm","snrd_total","snrd_wm","summary_bg_k","summary_bg_mad","summary_bg_mean","summary_bg_median","summary_bg_n",
#"summary_bg_p05","summary_bg_p95","summary_bg_stdv","summary_csf_k","summary_csf_mad","summary_csf_mean","summary_csf_median","summary_csf_n","summary_csf_p05","summary_csf_p95","summary_csf_stdv","summary_fg_k","summary_fg_mad","summary_fg_mean",
#"summary_fg_median","summary_fg_n","summary_fg_p05","summary_fg_p95","summary_fg_stdv","summary_gm_k","summary_gm_mad","summary_gm_mean","summary_gm_median","summary_gm_n","summary_gm_p05","summary_gm_p95","summary_gm_stdv","summary_wm_k","summary_wm_mad",
#"summary_wm_mean","summary_wm_median","summary_wm_n","summary_wm_p05","summary_wm_p95","summary_wm_stdv","tpm_overlap_csf","tpm_overlap_gm","tpm_overlap_wm","tsnr","wm2max"]

In [9]:
# Load in your own data # 

# This should be a .csv or .tsv file outputted from MRIQC on your own data
# This will return a pandas dataframe of the MRIQC data from your experiment

userdf = load_groupfile(group_file)
# userdf.head()
userdf.shape




(598, 45)

In [10]:
#userdf = userdf.loc[0:10,:]
#userdf.shape

In [11]:
# Load and filter data from the API # 

# Figure out which to get from modality arg #
T1apicsv = os.path.join(here, 'demo_api', 'T1w_demo.csv')
T2apicsv = os.path.join(here, 'demo_api', 'T2w_demo.csv')
boldapicsv = os.path.join(here, 'demo_api', 'bold_demo.csv')

if modality == 'T1w':
    api_file = T1apicsv
elif modality == 'T2w':
    api_file = T1apicsv
elif modality == 'bold':
    api_file = boldapicsv

# This will return a pandas dataframe with data from all scans of the given scan type
# with the given parameters 

apidf = pd.read_csv(api_file)
if not filter_list == []:
    filtered_apidf = filterIQM(apidf,filter_list)
else:
    filtered_apidf = apidf

# apidf.head()
print(apidf.shape)
# filtered_apidf.head()
print(filtered_apidf.shape)
#print(list(filtered_apidf))

(1000, 77)
(834, 77)


In [14]:
# Merge dataframes # 

# Takes the user data and API data and merges it into one dataframe 
# This will return a single pandas dataframe with the local data and API data merged, with a "group" measure to allow for a "groupby" 
# this needs to be updated with actual function name and information about how to use  

vis_ready_df = merge_dfs(userdf.copy(), filtered_apidf.copy())
#print(vis_ready_df.head())
#print(vis_ready_df.tail())
vis_ready_df.shape



v = make_vio_plot(vis_ready_df,IQM_to_plot,"", "This study",outliers=True)
#after_preproc = v[2]
widgets.VBox([v[0],v[1]])

Loading in dataframe...
Loading variables: ['fd_mean', 'fd_num', 'fd_perc']
Loading in data descriptors...
